In [56]:
# Calibrate piecewise-constant implied vol per maturity for the SPX file.
import pandas as pd
import numpy as np
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq

file_path = "spx_quotedata20220309_all.xlsx"
S0 = 4277.88
TRADE_DATE = pd.to_datetime("2022-03-09")
MIN_DAYS = 6

# Load data
df = pd.read_excel(file_path, sheet_name=0)
df.columns = [c.strip() for c in df.columns]
first_expiration = df.sort_values("Expiration Date")["Expiration Date"].iloc[0]

# Identify columns
exp_col = "Expiration Date"
strike_col = next(c for c in df.columns if c.lower().startswith("strike"))
call_bid_col = "Bid"
call_ask_col = "Ask"
call_vol_col = "Volume"
put_bid_col = "Bid.1" if "Bid.1" in df.columns else next(c for c in df.columns if c.lower().startswith("bid") and c!="bid")
put_ask_col = "Ask.1" if "Ask.1" in df.columns else next(c for c in df.columns if c.lower().startswith("ask") and c!="ask")
put_vol_col = "Volume.1" if "Volume.1" in df.columns else next(c for c in df.columns if c.lower().startswith("volume") and c!="volume")

# Parse expiry and compute T
def parse_exp(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        try:
            return pd.to_datetime(x, format="%a %b %d %Y")
        except Exception:
            return pd.NaT

df[exp_col] = df[exp_col].apply(parse_exp)
df["T_days"] = (df[exp_col] - TRADE_DATE).dt.days
df["T_years"] = df["T_days"] / 365.0

# numeric and mids
df[strike_col] = pd.to_numeric(df[strike_col], errors="coerce")
for c in [call_bid_col, call_ask_col, call_vol_col, put_bid_col, put_ask_col, put_vol_col]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["C_mid"] = (df[call_bid_col] + df[call_ask_col]) / 2.0
df["P_mid"] = (df[put_bid_col] + df[put_ask_col]) / 2.0
df[call_vol_col] = df[call_vol_col].fillna(0)
df[put_vol_col] = df[put_vol_col].fillna(0)

# Filter
mask = (df[call_vol_col] > 0) & (df[put_vol_col] > 0) & (df["T_days"] > MIN_DAYS)
df_filtered = df[mask].copy()

# Compute B and D per maturity by OLS (as earlier)
def fit_B_D_for_group(g):
    y = (g["C_mid"] - g["P_mid"]).to_numpy()
    X = np.column_stack([np.full(len(g), S0), -g[strike_col].to_numpy()])
    beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
    D_hat, B_hat = beta[0], beta[1]
    return D_hat, B_hat

groups = df_filtered.groupby("T_days", sort=True)
term_rows = []
for T_days, g in groups:
    D_hat, B_hat = fit_B_D_for_group(g)
    T_years = float(T_days)/365.0
    # avoid nonpositive
    r_bar = -np.log(B_hat)/T_years if (B_hat>0 and T_years>0) else np.nan
    q_bar = -np.log(D_hat)/T_years if (D_hat>0 and T_years>0) else np.nan
    term_rows.append({"T_days": T_days, "T_years": T_years, "D_0T": D_hat, "B_0T": B_hat, "r_bar": r_bar, "q_bar": q_bar})

term_struct = pd.DataFrame(term_rows).sort_values("T_days").reset_index(drop=True)

# Black-Scholes call price (European, continuous q)
def bs_call_price(S, K, T, r, q, sigma):
    if T<=0 or sigma<=0:
        return max(0.0, S*exp(-q*T) - K*exp(-r*T))
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    return S*exp(-q*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)

# implied vol via brentq on price difference
def implied_vol_from_price(price, S, K, T, r, q, tol=1e-8):
    # bounds
    lower = 1e-6
    upper = 5.0
    # If price equals intrinsic (or price very close to intrinsic), return tiny vol
    intrinsic = max(0.0, S*exp(-q*T) - K*exp(-r*T))
    # Price cannot be below intrinsic; if it is (due to quotes), set to intrinsic + epsilon
    if price < intrinsic:
        price = intrinsic
    # define f(sigma) = bs_price - market_price
    def f(sig):
        return bs_call_price(S, K, T, r, q, sig) - price
    # check sign at bounds
    try:
        if f(lower) * f(upper) > 0:
            # try widen upper
            upper = 20.0
            if f(lower) * f(upper) > 0:
                return np.nan
        vol = brentq(f, lower, upper, xtol=tol, rtol=tol, maxiter=200)
        return vol
    except Exception:
        return np.nan

# For each maturity pick strike closest to forward F = S0 * exp((r-q)*T)
results = []
for _, row in term_struct.iterrows():
    T_days = int(row["T_days"])
    T = float(row["T_years"])
    D0T = row["D_0T"]
    B0T = row["B_0T"]
    r = row["r_bar"]
    q = row["q_bar"]
    # find group
    g = df_filtered[df_filtered["T_days"]==T_days].copy()
    if g.empty:
        continue
    # compute forward from r,q if available, else from parity using mid prices averaged across strikes
    if np.isfinite(r) and np.isfinite(q):
        F = S0 * np.exp((r - q) * T)
    else:
        # use average parity-based forward: (C_mid - P_mid)/D + K, median across strikes to reduce noise
        F_vals = (g["C_mid"] - g["P_mid"]) / (D0T if D0T!=0 else 1e-12) + g[strike_col]
        F = float(np.nanmedian(F_vals))
    # choose strike closest to F
    g["dist"] = np.abs(g[strike_col] - F)
    g_sorted = g.sort_values("dist")
    chosen = g_sorted.iloc[0]
    K_star = float(chosen[strike_col])
    C_mid = float(chosen["C_mid"])
    P_mid = float(chosen["P_mid"])
    # implied vol using r,q
    vol = implied_vol_from_price(C_mid, S0, K_star, T, r if np.isfinite(r) else 0.0, q if np.isfinite(q) else 0.0)
    results.append({
        "Expiration": chosen[exp_col],
        "T_days": T_days,
        "T_years": T,
        "D_0T": D0T,
        "B_0T": B0T,
        "r_bar": r,
        "q_bar": q,
        "Forward": F,
        "K_star": K_star,
        "C_mid": C_mid,
        "P_mid": P_mid,
        "ImpliedVol": vol
    })

vol_table = pd.DataFrame(results).sort_values("T_days").reset_index(drop=True)
# Save CSV
out_csv = "spx_implied_vol_term_structure.csv"
vol_table.to_csv(out_csv, index=False)

vol_table.head(50)

,Expiration,T_days,T_years,D_0T,B_0T,r_bar,q_bar,Forward,K_star,C_mid,P_mid,ImpliedVol
0,2022-03-16,7,0.019178,0.998800,0.999692,0.016054,0.062634,4274.060234,4275.0,71.45,72.40,0.304654
1,2022-03-18,9,0.024658,0.998203,0.999114,0.035956,0.072942,4273.980350,4275.0,82.15,83.30,0.308991
2,2022-03-21,12,0.032877,0.998710,0.999711,0.008785,0.039275,4273.593929,4275.0,87.60,89.00,0.285707
3,2022-03-23,14,0.038356,0.999414,1.000426,-0.011115,0.015282,4273.550866,4270.0,98.10,94.60,0.288485
4,2022-03-25,16,0.043836,0.998669,0.999736,0.006020,0.030376,4273.315217,4275.0,102.50,104.20,0.289585
5,2022-03-28,19,0.052055,0.998323,0.999392,0.011680,0.032252,4273.301448,4275.0,106.95,108.75,0.277301
6,2022-03-30,21,0.057534,0.997171,0.998333,0.028990,0.049244,4272.897884,4270.0,116.25,113.40,0.281379
7,2022-03-31,22,0.060274,0.998184,0.999418,0.009652,0.030159,4272.595649,4275.0,116.45,118.90,0.281276
8,2022-04-01,23,0.063014,0.998295,0.999494,0.008037,0.027075,4272.751149,4275.0,119.55,121.95,0.282139
9,2022-04-04,26,0.071233,0.997935,0.999163,0.011754,0.029018,4272.622413,4260.0,132.30,119.50,0.277437
